https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

In [13]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import random

In [14]:
# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [16]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir='./ExDark', label_dir='./ExDarkAnnotation', transform=None):
        '''
        Args:
            img_dir: image root directory containing the different class folders ie 'img_dir/class1/image1'
            label_dir: label root directory containing the different class folders ie 'label_dir/class1/label1'
            transform: transforms to apply to the input image
        '''
        self.label_dir = label_dir
        self.img_dir = img_dir
        self.transform = transform
        self.imagelist = [] #list of lists containing [image_name, class_name]
        # example of self.imagelist[12] = ['2015_01863.png', 'Bottle']
        self.class_to_num = {} #maps class name to numeric value ie {'Bottle':1,'Bicycle':2}
        
        #go through each class folder
        class_counter = 1
        for class_name in os.listdir(img_dir):
            self.class_to_num[class_name] = class_counter
            class_counter += 1
            classpath = img_dir+'/'+class_name # './img_dir/class_name'
            
            #go through each image in the class folder
            for img in os.listdir(classpath): 
                self.imagelist.append([img, class_name])

    def __len__(self):
        return len(self.imagelist)

    def __getitem__(self, idx):
        '''
        Returns:
            image: (PIL or a torch.Tensor) input image
            target: (dict) label dictionary required for faster R-CNN
        '''
        image_name, class_folder = self.imagelist[idx]
        img_path = os.path.join(self.img_dir,class_folder,image_name)
        target_path = os.path.join(self.label_dir,class_folder,image_name+'.txt')
        
        image = Image.open(img_path).convert("RGB")
        
        if self.transform is not None:
            image = self.transform(image)
            image = image.to(dtype=torch.float32)
        
        boxes = []
        labels = []
        with open(target_path,'r') as f:
            for line in f:
                if line[0] == '%':
                    continue
                else:
                    line = line.split()
                    classlabel = line[0]
                    labels.append(self.class_to_num[classlabel])
                    xmin = int(line[1])
                    ymin = int(line[2])
                    xmax = xmin + int(line[3])
                    ymax = ymin + int(line[4])
                    boxes.append([xmin, ymin, xmax, ymax]) #bounding box coords
        
        assert len(boxes) == len(labels)
        
        #turn everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        
        return image, target

In [50]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 13  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /tmp/xdg-cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [24]:
input_transform = T.Compose([T.PILToTensor()])
dataset = CustomImageDataset(img_dir='./ExDark',label_dir='./ExDarkAnnotation',transform=input_transform)
dataset_test = CustomImageDataset(img_dir='./ExDark',label_dir='./ExDarkAnnotation',transform=input_transform)

#split the dataset into train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-1411])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-1411:])

In [3]:
batch_size = 32
iterations = int(len(dataset)/batch_size) #chose 1411 and batch_size=32 so we can divide dataset by batch_size evenly
num_epochs = 1

optimizer = torch.optim.SGD(model.parameters(), lr=0.005,
                            momentum=0.9, weight_decay=0.0005)


for epoch in range(num_epochs):
    for it in range(iterations):
        #get batch data
        for b in range(batch_size):
            
        

cpu
